# Libraries

In [3]:
# general
import pandas as pd
import numpy as np

# Machine Learning
import torch
import torch.nn as nn
import torch.nn.functional as F

%store -r predictions labels crps_weights

# CRPS

continouos ranked probability score

In [4]:
def crps(forecast, observations, weights):
    """
    Args:
    forecast (torch.Tensor): Forecasts from the model (ensemble) with shape [1, seq_len].
    observations (torch.Tensor): Observed values with shape [seq_len].
    weights (torch.Tensor): Corresponding weights for the CRPS scores, derived from sparse attention, with shape [1, seq_len, seq_len].

    Returns:
    float: Weighted mean of the CRPS for all forecasts.
    """
    forecast = forecast.squeeze(0)  # Adjusting forecast shape: [64]
    weights = weights.mean(dim=-1).squeeze(0)  # Assuming averaging is the method to obtain weights: [64]
    
    # Sorting the forecasts
    sorted_forecast, _ = torch.sort(forecast, dim=0)
    observations = observations.unsqueeze(0)  # [1, 64] for broadcasting

    # Cumulative sum of sorted forecasts
    cumsum_forecast = torch.cumsum(sorted_forecast, dim=0) / forecast.size(0)

    # Calculating CRPS
    indicator = (sorted_forecast > observations).float()
    differences = (cumsum_forecast - indicator) ** 2
    weighted_differences = differences * weights  # Apply weights to the differences
    crps = weighted_differences.mean()  # Taking mean across all weighted differences

    return crps.item()  # Returning as a Python float

In [5]:
crps(forecast=predictions, observations=labels, weights=crps_weights)

RuntimeError: The size of tensor a (64) must match the size of tensor b (512) at non-singleton dimension 1

In [ ]:
import torch
import torch.nn as nn

class CRPSLoss(nn.Module):
    def __init__(self):
        super(CRPSLoss, self).__init__()

    def forward(self, pred_params, observations):
        """
        Compute the CRPS loss.
        
        Args:
        - pred_params: Parameters predicted by the model that define the distribution of X. 
                       This could be the mean and standard deviation if X is assumed to be Gaussian.
        - observations: Actual observed values.
        
        Returns:
        - crps_loss: The CRPS loss value.
        """
        # For demonstration, let's assume pred_params represents the mean of X,
        # and X follows a Gaussian distribution with a known standard deviation.
        mu = pred_params  # Model predictions
        sigma = torch.full_like(mu, 1.0)  # Assuming a fixed standard deviation for simplicity
        
        # First term: E[|X-x|]
        # Using a sampled approach or analytic solution if available.
        # This is a placeholder for the actual computation.
        term1 = torch.abs(mu - observations).mean()

        # Second term: 0.5 * E[|X-X*|]
        # For a Gaussian distribution, E[|X-X*|] is sqrt(2/pi) * sigma if X ~ N(mu, sigma^2)
        term2 = 0.5 * (torch.sqrt(torch.tensor(2 / torch.pi)) * sigma).mean()

        # Compute CRPS loss
        crps_loss = term1 - term2
        return crps_loss

# Example usage
crps_loss = CRPSLoss()

# pred_params should be the predicted parameters of the distribution of X, e.g., mean of a Gaussian.
# observations should be the actual observed values.
pred_params = torch.randn(10, 1)  # Example predictions
observations = torch.randn(10, 1)  # Example observations

loss = crps_loss(pred_params, observations)
print(loss)


tensor(31.6295)


In [8]:
print(f"labels: {labels.shape}")
print(f"predictions: {predictions.shape}")
print(f"weights: {crps_weights.shape}")

labels: torch.Size([512])
predictions: torch.Size([1, 64])
weights: torch.Size([1, 64, 64])
